In [1]:
import pandas as pd
import datetime

# Import and explore the "spend" file

In [2]:
#Import the spend file as a Pandas dataframe
spend_df = pd.read_csv('spend.csv.gz', compression='gzip', header=0, sep=',', quotechar='"')

In [3]:
#Let's see what we're dealing with
spend_df.head()

,account,date,amount
0,5e0a5eb5575517efdddb2799c90e2e9b,1502841600,15.144020
1,a04e063493c55bc6cecd9056712e9e47,102841600,730.070453
2,ff6cc8cadcb7457f9ddb2dd238a13733,1502841600,3651.412248
3,324a05f1f917b8a0af83a54e55a1ef63,1502841600,113.219243
4,b9abff45dc08172b556cb10b7c3c2252,1502841600,697.224210


In [4]:
#Let's see what data types these are
spend_df.dtypes

account     object
date         int64
amount     float64
dtype: object

In [5]:
#Check for NaN values
spend_df.isna().sum()

account    0
date       0
amount     0
dtype: int64

In [6]:
#let's try to get a basic summary of this data
spend_df.describe()

,date,amount
count,1.934135e+06,1.934135e+06
mean,1.531318e+09,3.018366e+02
std,1.346509e+07,1.121577e+03
min,1.028416e+08,-6.051673e+02
25%,1.520986e+09,1.532719e+01
50%,1.532909e+09,6.613464e+01
75%,1.542499e+09,2.349944e+02
max,1.552608e+09,3.073705e+05


In [7]:
#the scientific notation makes that hard to digest so let's get some simpler numbers
print(f"Rows: {spend_df['amount'].count()}")
print(f"Mean Amount: {spend_df['amount'].mean()}")
print(f"Median Amount: {spend_df['amount'].median()}")

Rows: 1934135
Mean Amount: 301.836556328393
Median Amount: 66.1346409112299


In [8]:
#Convert the 10 digit unix date to datetime in the df, then preview the result
spend_df['date'] = pd.to_datetime(spend_df['date'], unit='s')
spend_df.head()

,account,date,amount
0,5e0a5eb5575517efdddb2799c90e2e9b,2017-08-16 00:00:00,15.144020
1,a04e063493c55bc6cecd9056712e9e47,1973-04-05 07:06:40,730.070453
2,ff6cc8cadcb7457f9ddb2dd238a13733,2017-08-16 00:00:00,3651.412248
3,324a05f1f917b8a0af83a54e55a1ef63,2017-08-16 00:00:00,113.219243
4,b9abff45dc08172b556cb10b7c3c2252,2017-08-16 00:00:00,697.224210


In [9]:
#That 1973 date (index #1) looks suspicious. Let's see if there are any other dates that old.
spend_df['date'].nsmallest(5)

1   1973-04-05 07:06:40
0   2017-08-16 00:00:00
2   2017-08-16 00:00:00
3   2017-08-16 00:00:00
4   2017-08-16 00:00:00
Name: date, dtype: datetime64[ns]

In [10]:
#This inconsistency appears to be the result of a digit missing from unix timestamp.
#The inconsistent record should be dropped:
spend_df_cleaned = spend_df.drop(1)

In [11]:
#Check for outliers in the 'amount' column by looking at largest and smallest values
print(spend_df_cleaned['amount'].nsmallest(5))
print('----------------------------')
print(spend_df_cleaned['amount'].nlargest(5))

1934132   -6.051673e+02
1663987    1.649581e-26
42278      5.234396e-23
1811362    2.177845e-16
1179533    1.118548e-15
Name: amount, dtype: float64
----------------------------
1085481    307370.457700
1125984    307370.457700
495209     244192.048700
754373     188632.196857
756878     178689.898916
Name: amount, dtype: float64


In [12]:
#There is only one negative number in a set of over a million
#It should be dropped:
spend_df_cleaned = spend_df_cleaned.drop(1934132)

#check that it is gone
spend_df_cleaned['amount'].nsmallest(5)

1663987    1.649581e-26
42278      5.234396e-23
1811362    2.177845e-16
1179533    1.118548e-15
1007627    5.510904e-13
Name: amount, dtype: float64

In [13]:
#Now we can find what date range this dataset actually covers
f"The 'spend' file contains records from {spend_df_cleaned['date'].min()} to {spend_df_cleaned['date'].max()}"

"The 'spend' file contains records from 2017-08-16 00:00:00 to 2019-03-15 00:00:00"

In [14]:
#How many unique accounts are in this data?
unique_accounts = spend_df_cleaned.drop_duplicates(subset=['account'], keep='first').count()[0]
unique_accounts

10001

# Repeat exploration for the "counts" file

In [15]:
#Import the counts file as a Pandas dataframe
counts_df = pd.read_csv('counts.csv.gz', compression='gzip', header=0, sep=',', quotechar='"')

In [16]:
counts_df.head()

,account,date,count
0,5e0a5eb5575517efdddb2799c90e2e9b,2017-08-16,56
1,a04e063493c55bc6cecd9056712e9e47,2017-08-16,180
2,ff6cc8cadcb7457f9ddb2dd238a13733,2017-08-16,432
3,324a05f1f917b8a0af83a54e55a1ef63,2017-08-16,50
4,b9abff45dc08172b556cb10b7c3c2252,2017-08-16,135


In [17]:
#Check the datatypes
counts_df.dtypes

account    object
date       object
count       int64
dtype: object

In [18]:
#Checking for NaN values
counts_df.isna().sum()

account    0
date       0
count      0
dtype: int64

In [19]:
counts_df.describe()

,count
count,1.934028e+06
mean,4.145873e+01
std,7.499475e+01
min,1.000000e+01
25%,1.500000e+01
50%,2.400000e+01
75%,4.400000e+01
max,5.643000e+03


In [20]:
#the scientific notation makes that hard to digest so let's get some simpler numbers
print(f"Rows: {counts_df['count'].count()}")
print(f"Mean Count: {counts_df['count'].mean()}")
print(f"Median Count: {counts_df['count'].median()}")

Rows: 1934028
Mean Count: 41.458726554114
Median Count: 24.0


In [21]:
#convert the 'date' column to a datetime dtype
counts_df['date'] = pd.to_datetime(counts_df['date'])
counts_df['date'].nsmallest()

0   2017-08-16
1   2017-08-16
2   2017-08-16
3   2017-08-16
4   2017-08-16
Name: date, dtype: datetime64[ns]

In [22]:
#Find what date range this dataset covers
f"The 'counts' file contains records from {counts_df['date'].min()} to {counts_df['date'].max()}"

"The 'counts' file contains records from 2017-08-16 00:00:00 to 2019-03-15 00:00:00"

In [23]:
#Check for outliers in the 'count' column by looking at largest and smallest values
print(counts_df['count'].nsmallest(5))
print('----------------------------')
print(counts_df['count'].nlargest(5))

183    10
214    10
324    10
720    10
798    10
Name: count, dtype: int64
----------------------------
434635    5643
389999    5313
415974    5170
401479    4686
401324    4664
Name: count, dtype: int64


In [24]:
#How many unique accounts are in this data?
unique_accounts2 = counts_df.drop_duplicates(subset=['account'], keep='first').count()[0]
unique_accounts2

9999

# Merge Datasets and explore how they fit together

In [25]:
#It looks like both datasets have a dual primary key of 'account' and 'date' so we will merge on those
#Use outer join so we can see which rows had no match
merged_df = pd.merge(spend_df_cleaned, counts_df, on=['account', 'date'], how='outer')
merged_df.head()

,account,date,amount,count
0,5e0a5eb5575517efdddb2799c90e2e9b,2017-08-16,15.144020,56.0
1,ff6cc8cadcb7457f9ddb2dd238a13733,2017-08-16,3651.412248,432.0
2,324a05f1f917b8a0af83a54e55a1ef63,2017-08-16,113.219243,50.0
3,b9abff45dc08172b556cb10b7c3c2252,2017-08-16,697.224210,135.0
4,e6ff738c8dd54092b5b1de683fc71dc0,2017-08-16,41.867847,76.0


In [26]:
merged_df.count()

account    1934137
date       1934137
amount     1934133
count      1934028
dtype: int64

In [28]:
#How many unique accounts between both datasets?
merged_df.drop_duplicates(subset=['account'], keep='first').count()[0]

10001

In [29]:
#Check for NaN values
merged_df.isna().sum()

account      0
date         0
amount       4
count      109
dtype: int64

In [30]:
#Create a df for exploring rows with NaNs to see if we can tell why
nulls_df = merged_df[merged_df.isnull().any(axis=1)]
nulls_df

,account,date,amount,count
328532,04aa55b1463f98f26d220ffb618b969b,2018-10-07,152.218665,NaN
387162,04aa55b1463f98f26d220ffb618b969b,2018-11-06,54.132141,NaN
422584,04aa55b1463f98f26d220ffb618b969b,2018-11-14,665.369130,NaN
438909,04aa55b1463f98f26d220ffb618b969b,2018-12-05,54.190954,NaN
441643,04aa55b1463f98f26d220ffb618b969b,2018-11-26,13.351277,NaN
448395,04aa55b1463f98f26d220ffb618b969b,2018-12-07,153.162176,NaN
449510,04aa55b1463f98f26d220ffb618b969b,2018-12-13,63.522312,NaN
510397,04aa55b1463f98f26d220ffb618b969b,2019-02-05,83.585540,NaN
515736,04aa55b1463f98f26d220ffb618b969b,2019-01-25,63.522312,NaN
533030,04aa55b1463f98f26d220ffb618b969b,2019-02-06,211.309084,NaN


In [31]:
#check whether account '04aa55b1463f98f26d220ffb618b969b' ever appears in the counts data
check_acct = counts_df.loc[counts_df['account']=='04aa55b1463f98f26d220ffb618b969b']
check_acct

,account,date,count


In [32]:
#108 of the NaNs occured because acct '04aa55b1463f98f26d220ffb618b969b' only appeared in the spend dataset
#1 of the NaNs occured because row 1934129 is missing 2 digits in acct number '9624306955fe5efaeb3b718c303f2'
#4 of the NaNs appear to be one-off rows that had no match in both datasets
#In any case, dropping NaNs will aleviate these issues while losing relatively little data:
merged_df_cleaned = merged_df.dropna()
merged_df_cleaned.head()

,account,date,amount,count
0,5e0a5eb5575517efdddb2799c90e2e9b,2017-08-16,15.144020,56.0
1,ff6cc8cadcb7457f9ddb2dd238a13733,2017-08-16,3651.412248,432.0
2,324a05f1f917b8a0af83a54e55a1ef63,2017-08-16,113.219243,50.0
3,b9abff45dc08172b556cb10b7c3c2252,2017-08-16,697.224210,135.0
4,e6ff738c8dd54092b5b1de683fc71dc0,2017-08-16,41.867847,76.0


In [33]:
#How many unique accounts are there after droping rows with NaNs?
merged_df_cleaned.drop_duplicates(subset=['account'], keep='first').count()[0]

9999

In [34]:
#This^ lines up with findings above

In [35]:
#Export the merged and cleaned dataset for future analysis
merged_df_cleaned.to_csv('accounts_data_merged.csv', index=False)